# Example deployment function


## Orig version

In [1]:
import pandas as pd

In [2]:
def dissonance(
    resp_dict,
    model_paths,
    classifier,
):
    """_summary_

    Args:
        resp_dict (_type_): response dictionary with keys as questions and values as subjects' responses
        model_paths (_type_): paths to qnet models used for computing dissonances
        question_order (_type_): order of questions used by qnets specified in `model_paths`
        classifier (_type_, optional): pretrained classifier used to compute malingering probability. Defaults to None.

    Returns:
        _type_: dissonance(s) of subjects (and malingering prob, if classifier is provided)
    """
    import numpy as np
    from quasinet.qnet import load_qnet

    def _diss_linear(s, qnet, missing_value=0):
        diss = list()
        Ds = qnet.predict_distributions(s)

        for i in range(len(s)):
            if s[i] != "":
                if s[i] in Ds[i].keys():
                    diss.append(1 - Ds[i][s[i]] / np.max(list(Ds[i].values())))
                elif s[i] == "missing":
                    diss.append(missing_value)
                else:
                    diss.append(1)

        return np.array(diss)

    def _actual_sample_dissonance(
        data_sample, diss_models, diss_fcn, order, length, missing_value=0
    ):
        if order is None:
            order = range(length)

        sample = np.full(length, "", dtype="<U21")

        diss = [list() for model in diss_models]

        for i in order:
            if data_sample[i] == "":
                sample[i] = "missing"
            else:
                sample[i] = data_sample[i]
            for d, model in zip(diss, diss_models):
                d.append(diss_fcn(sample, model, missing_value))

        return sample, diss

    def _all_actual_samples_dissonance(
        data_samples, diss_models, diss_fcn, order, length, missing_value=0
    ):
        samples = list()
        dissonances = list()

        for data_sample in data_samples:
            samp, diss = _actual_sample_dissonance(
                data_sample, diss_models, diss_fcn, order, length, missing_value
            )
            samples.append(samp)
            dissonances.append(diss)

        return samples, dissonances

    def _dissonance_data_at_question(dissonances, questions_asked):
        return np.array(
            [np.hstack([d[questions_asked - 1] for d in diss]) for diss in dissonances]
        )

    diss_models = [
        load_qnet(
            model_path,
            gz=True,
        )
        for model_path in model_paths
    ]
    resp_df = pd.DataFrame(resp_dict).T

    resp_fmtd = (
        resp_df[diss_models[0].feature_names]
        .fillna(-9)
        .astype(int)
        .replace(-9, "")
        .to_numpy()
    )

    resp_diss = _all_actual_samples_dissonance(
        resp_fmtd,
        diss_models,
        _diss_linear,
        range(resp_fmtd.shape[1]),
        resp_fmtd.shape[1],
    )[1]

    resp_new = pd.DataFrame(_dissonance_data_at_question(resp_diss, resp_fmtd.shape[1]))

    proba = classifier.predict_proba(resp_new)

    output_dict = dict.fromkeys(
        resp_dict.keys(),
    )
    for key, val in zip(output_dict.keys(), proba[:, 1]):
        output_dict[key] = val

    return output_dict

In [3]:
resp_dict = pd.read_pickle("global-data-example-dict.pkl")
model_paths = [
    f"models/global/random_order_{model}_model_0.joblib.gz"
    for model in ["full"]  # , "neg", "pos"]
]
classifier = pd.read_pickle(f"classifiers/global/runif-classifier-{346}.pkl")

In [4]:
dissonance(resp_dict, model_paths, classifier)

{0: 0.0,
 1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.07}

## Updated version

In [5]:
import pandas as pd
import numpy as np
from quasinet.qnet import load_qnet

VERBOSE = 0


def _diss_linear(s, qnet, missing_response="", missing_diss_value=0):
    Ds = qnet.predict_distributions(s)
    diss_values = []

    for i in range(len(s)):
        prob = float(Ds[i].get(str(s[i]), np.max(list(Ds[i].values()))))
        max_prob = max(Ds[i].values())
        diss_value = 1 - prob / max_prob if max_prob != 0 else 0
        if VERBOSE:
            print(i, "-->" + s[i] + "<", Ds[i], prob, max_prob, diss_value)
        diss_values.append(diss_value)

    if VERBOSE:
        print(diss_values)

    return diss_values


def dissonance(patients_responses, problem_type, model_path, classifier_path):
    """Computes dissonance values and malingering

    Args:
        patients_responses (_type_): dict of patients' responses
        problem_type (_type_): one of "global", "ptsd", "bond-court"
        model_path (_type_): path(s) to model(s) to use for computing dissonances ("full" for Qnet trained on all samples, "pos" for Qnet trained on positive samples, "neg" for Qnet trained on negative samples )
        classifier_path (_type_): path to pretrained classifier where positive class represents malingerers

    Returns:
        _type_: malingering risk (defined as class probability of the malingering class)
    """
    model = load_qnet(model_path)
    classifier = pd.read_pickle(classifier_path)

    all_data_samples = []

    for patient_response in patients_responses:
        for patient_id, responses in patient_response.items():
            resp_df = pd.DataFrame([responses], columns=model.feature_names)
            data_samples = (
                resp_df.fillna("")  # Replace missing values with empty strings
                .astype(str)  # Convert all values to strings
                .values
            )
            all_data_samples.append(data_samples[0])

    diss_values = [_diss_linear(sample, model) for sample in all_data_samples]
    proba = classifier.predict_proba(diss_values)

    # Create output dictionary
    output_dict = {}
    for idx, patient_response in enumerate(patients_responses):
        for patient_id in patient_response.keys():
            output_dict[patient_id] = proba[idx, 1]  # Assign the correct probability

    return output_dict


# Example usage
problem_type = "global/runif"
model_path = f"models/global/random_order_full_model_0.joblib.gz"
classifier_path = f"classifiers/{problem_type}-classifier-{346}.pkl"


patients_responses = [
    {
        "0": {
            "20": "1",
            "31": 2.0,
            "36": 1.0,
            "51": 2.0,
            "60": 1.0,
            "75": 1.0,
            "85": 1.0,
            "115": 1.0,
            "123": 2.0,
            "147": 2.0,
            "150": 3.0,
            "179": 2.0,
            "312": 2.0,
            "355": 2.0,
            "436": 4.0,
            "572": 3.0,
            "719": 2.0,
            "737": 2.0,
            "747": 2.0,
            "817": 2.0,
            "855": 2.0,
            "938": 1.0,
            "983": 2.0,
            "1053": 2.0,
            "1060": 2.0,
            "1069": 1.0,
            "1132": 2.0,
            "4201": 1.0,
            "4203": 1,
            "4247": 1.0,
            "4250": 2.0,
            "4251": 2.0,
            "4254": 2.0,
            "4255": 1,
            "4262": 2.0,
            "4278": 1,
            "4280": 1,
            "4287": 1,
            "4289": 1,
            "4439": 1.0,
            "4441": 1.0,
            "4442": 2.0,
            "4452": 1.0,
            "4453": 0,
            "4454": 0,
            "4455": 0,
            "4456": 0.0,
            "4525": 1,
            "4527": 1,
            "4529": 3.0,
            "4531": 1.0,
            "4532": 2.0,
            "4545": 3.0,
            "4553": 1.0,
            "4555": 1,
            "4571": 1.0,
            "4574": 2.0,
            "4587": 1.0,
        }
    },
    {
        "9": {
            "20": 2.0,
            "31": 1.0,
            "36": 1.0,
            "46": 1.0,
            "51": 1.0,
            "53": 1.0,
            "61": 2.0,
            "62": 2.0,
            "76": 1.0,
            "85": 1.0,
            "109": 3.0,
            "110": 4.0,
            "111": 3.0,
            "123": 2.0,
            "149": 4.0,
            "210": 3.0,
            "255": 2.0,
            "576": 1.0,
            "603": 4.0,
            "840": 2.0,
            "841": 2.0,
            "855": 2.0,
            "881": 2.0,
            "927": 2.0,
            "928": 2.0,
            "936": 2.0,
            "1009": 3.0,
            "1041": 3.0,
            "1080": 3.0,
            "1125": 2.0,
            "4203": 1,
            "4247": 1.0,
            "4248": 4.0,
            "4250": 2.0,
            "4251": 3.0,
            "4255": 3,
            "4262": 1.0,
            "4278": 1,
            "4280": 1,
            "4287": 1,
            "4289": 1,
            "4355": 3.0,
            "4439": 3.0,
            "4441": 3.0,
            "4451": 2.0,
            "4452": 3.0,
            "4453": 0,
            "4454": 0,
            "4455": 0,
            "4456": 0.0,
            "4525": 2,
            "4527": 1,
            "4554": 2.0,
            "4555": 3,
            "4586": 1.0,
            "4587": 3.0,
            "4590": 1.0,
            "4596": 3.0,
        }
    },
    {"PHX1": {"20": "1", "31": 1.0}},
]

processed_responses = [
    {
        patient_id: {
            question_id: str(int(response))
            for question_id, response in patient_responses.items()
        }
        for patient_id, patient_responses in patient.items()
    }
    for patient in patients_responses
]


result = dissonance(processed_responses, problem_type, model_path, classifier_path)
print(result)

{'0': 0.0, '9': 0.07, 'PHX1': 0.0}


### Global

In [6]:
patients_responses = (
    pd.read_csv("../data/gibbons_global/gibbons_global.csv")
    .fillna(-9)
    .astype(int)
    .replace(-9, "")
    .to_dict("index")
)
processed_responses = [
    {patient: patients_responses[patient]} for patient in patients_responses
]


problem_type = "global/runif"
model_path = f"models/global/random_order_full_model_0.joblib.gz"
classifier_path = f"classifiers/{problem_type}-classifier-{346}.pkl"
dissonance(processed_responses, problem_type, model_path, classifier_path)

{0: 0.0,
 1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.07,
 18: 0.0,
 19: 0.01,
 20: 0.01,
 21: 0.0,
 22: 0.0,
 23: 0.0,
 24: 0.0,
 25: 0.0,
 26: 0.0,
 27: 0.0,
 28: 0.0,
 29: 0.0,
 30: 0.0,
 31: 0.0,
 32: 0.0,
 33: 0.01,
 34: 0.0,
 35: 0.0,
 36: 0.01,
 37: 0.0,
 38: 0.0,
 39: 0.0,
 40: 0.0,
 41: 0.0,
 42: 0.0,
 43: 0.01,
 44: 0.0,
 45: 0.0,
 46: 0.03,
 47: 0.0,
 48: 0.0,
 49: 0.0,
 50: 0.0,
 51: 0.0,
 52: 0.0,
 53: 0.0,
 54: 0.0,
 55: 0.0,
 56: 0.01,
 57: 0.0,
 58: 0.0,
 59: 0.0,
 60: 0.01,
 61: 0.0,
 62: 0.0,
 63: 0.0,
 64: 0.0,
 65: 0.0,
 66: 0.01,
 67: 0.0,
 68: 0.0,
 69: 0.0,
 70: 0.0,
 71: 0.0,
 72: 0.0,
 73: 0.0,
 74: 0.0,
 75: 0.0,
 76: 0.0,
 77: 0.0,
 78: 0.0,
 79: 0.0,
 80: 0.0,
 81: 0.0,
 82: 0.0,
 83: 0.0,
 84: 0.0,
 85: 0.0,
 86: 0.0,
 87: 0.0,
 88: 0.0,
 89: 0.01,
 90: 0.0,
 91: 0.0,
 92: 0.0,
 93: 0.0,
 94: 0.0,
 95: 0.0,
 96: 0.01,
 97: 0.0,
 98: 0.0,
 99: 0.0

In [7]:
patients_responses = (
    pd.read_csv("../data/gibbons_global/gibbons_global.csv")
    .fillna(-9)
    .astype(int)
    .replace(-9, "")
    .to_dict("index")
)
processed_responses = [
    {patient: patients_responses[patient]} for patient in patients_responses
]


problem_type = "global/expert"
model_path = f"models/global/random_order_full_model_0.joblib.gz"
classifier_path = f"classifiers/{problem_type}-classifier-{346}.pkl"
dissonance(processed_responses, problem_type, model_path, classifier_path)

{0: 0.0,
 1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.02,
 18: 0.0,
 19: 0.0,
 20: 0.0,
 21: 0.0,
 22: 0.0,
 23: 0.0,
 24: 0.0,
 25: 0.01,
 26: 0.0,
 27: 0.0,
 28: 0.02,
 29: 0.0,
 30: 0.0,
 31: 0.0,
 32: 0.0,
 33: 0.0,
 34: 0.0,
 35: 0.0,
 36: 0.02,
 37: 0.0,
 38: 0.0,
 39: 0.0,
 40: 0.0,
 41: 0.0,
 42: 0.0,
 43: 0.0,
 44: 0.0,
 45: 0.0,
 46: 0.0,
 47: 0.0,
 48: 0.0,
 49: 0.0,
 50: 0.0,
 51: 0.0,
 52: 0.0,
 53: 0.0,
 54: 0.0,
 55: 0.0,
 56: 0.0,
 57: 0.0,
 58: 0.01,
 59: 0.0,
 60: 0.0,
 61: 0.0,
 62: 0.0,
 63: 0.0,
 64: 0.0,
 65: 0.0,
 66: 0.02,
 67: 0.0,
 68: 0.0,
 69: 0.0,
 70: 0.0,
 71: 0.0,
 72: 0.0,
 73: 0.01,
 74: 0.0,
 75: 0.0,
 76: 0.0,
 77: 0.0,
 78: 0.0,
 79: 0.0,
 80: 0.0,
 81: 0.0,
 82: 0.0,
 83: 0.0,
 84: 0.0,
 85: 0.0,
 86: 0.0,
 87: 0.0,
 88: 0.0,
 89: 0.0,
 90: 0.0,
 91: 0.0,
 92: 0.0,
 93: 0.0,
 94: 0.0,
 95: 0.0,
 96: 0.0,
 97: 0.0,
 98: 0.0,
 99: 0.0,
 10

### PTSD

In [8]:
patients_responses = pd.read_csv("../data/ptsd/PTSD_cognet_test_processed.csv").to_dict(
    "index"
)
processed_responses = [
    {patient: patients_responses[patient]} for patient in patients_responses
]
problem_type = "ptsd/runif"
model_path = f"models/ptsd/random_order_full_model_0.joblib.gz"
classifier_path = f"classifiers/{problem_type}-classifier-{211}.pkl"
dissonance(processed_responses, problem_type, model_path, classifier_path)

{0: 0.16,
 1: 0.02,
 2: 0.17,
 3: 0.03,
 4: 0.07,
 5: 0.23,
 6: 0.08,
 7: 0.08,
 8: 0.02,
 9: 0.41,
 10: 0.04,
 11: 0.14,
 12: 0.07,
 13: 0.19,
 14: 0.09,
 15: 0.01,
 16: 0.12,
 17: 0.05,
 18: 0.22,
 19: 0.07,
 20: 0.17,
 21: 0.21,
 22: 0.17,
 23: 0.1,
 24: 0.04,
 25: 0.08,
 26: 0.04,
 27: 0.15,
 28: 0.02,
 29: 0.0,
 30: 0.0,
 31: 0.04,
 32: 0.17,
 33: 0.01,
 34: 0.02,
 35: 0.0,
 36: 0.31,
 37: 0.22,
 38: 0.14,
 39: 0.09,
 40: 0.03,
 41: 0.01,
 42: 0.07,
 43: 0.03,
 44: 0.12,
 45: 0.01,
 46: 0.36,
 47: 0.0,
 48: 0.05,
 49: 0.0,
 50: 0.12,
 51: 0.02,
 52: 0.32,
 53: 0.02,
 54: 0.0,
 55: 0.07,
 56: 0.02,
 57: 0.25,
 58: 0.13,
 59: 0.0,
 60: 0.46,
 61: 0.09,
 62: 0.03,
 63: 0.46,
 64: 0.04,
 65: 0.0,
 66: 0.06,
 67: 0.05,
 68: 0.0,
 69: 0.06,
 70: 0.01,
 71: 0.11,
 72: 0.07,
 73: 0.03,
 74: 0.03,
 75: 0.17,
 76: 0.04,
 77: 0.01,
 78: 0.04,
 79: 0.09,
 80: 0.02,
 81: 0.02,
 82: 0.02,
 83: 0.03,
 84: 0.02,
 85: 0.04,
 86: 0.42,
 87: 0.01,
 88: 0.07,
 89: 0.05,
 90: 0.23,
 91: 0.03,
 92: 0.0

In [9]:
patients_responses = pd.read_csv("../data/ptsd/PTSD_cognet_test_processed.csv").to_dict(
    "index"
)
processed_responses = [
    {patient: patients_responses[patient]} for patient in patients_responses
]
problem_type = "ptsd/expert"
model_path = f"models/ptsd/random_order_full_model_0.joblib.gz"
classifier_path = f"classifiers/{problem_type}-classifier-{211}.pkl"
dissonance(processed_responses, problem_type, model_path, classifier_path)

{0: 0.03,
 1: 0.03,
 2: 0.17,
 3: 0.05,
 4: 0.03,
 5: 0.14,
 6: 0.03,
 7: 0.09,
 8: 0.17,
 9: 0.41,
 10: 0.05,
 11: 0.02,
 12: 0.16,
 13: 0.14,
 14: 0.22,
 15: 0.01,
 16: 0.08,
 17: 0.2,
 18: 0.59,
 19: 0.1,
 20: 0.27,
 21: 0.2,
 22: 0.31,
 23: 0.25,
 24: 0.06,
 25: 0.1,
 26: 0.09,
 27: 0.03,
 28: 0.02,
 29: 0.07,
 30: 0.0,
 31: 0.05,
 32: 0.32,
 33: 0.05,
 34: 0.08,
 35: 0.0,
 36: 0.28,
 37: 0.11,
 38: 0.11,
 39: 0.29,
 40: 0.03,
 41: 0.0,
 42: 0.25,
 43: 0.02,
 44: 0.1,
 45: 0.04,
 46: 0.44,
 47: 0.0,
 48: 0.07,
 49: 0.0,
 50: 0.23,
 51: 0.01,
 52: 0.33,
 53: 0.05,
 54: 0.03,
 55: 0.08,
 56: 0.12,
 57: 0.43,
 58: 0.27,
 59: 0.01,
 60: 0.43,
 61: 0.01,
 62: 0.1,
 63: 0.49,
 64: 0.06,
 65: 0.0,
 66: 0.09,
 67: 0.05,
 68: 0.02,
 69: 0.1,
 70: 0.0,
 71: 0.13,
 72: 0.11,
 73: 0.1,
 74: 0.03,
 75: 0.3,
 76: 0.11,
 77: 0.02,
 78: 0.05,
 79: 0.11,
 80: 0.08,
 81: 0.03,
 82: 0.07,
 83: 0.14,
 84: 0.01,
 85: 0.07,
 86: 0.41,
 87: 0.04,
 88: 0.27,
 89: 0.07,
 90: 0.47,
 91: 0.11,
 92: 0.09,
 93

### Bond court

In [10]:
patients_responses = (
    pd.read_csv("../data/bondcourt/CCHHS_SUD_details.csv")
    .drop("Interview_ID", axis="columns")
    .fillna(-9)
    .astype(int)
    .replace(-9, "")
    .to_dict("index")
)
processed_responses = [
    {patient: patients_responses[patient]} for patient in patients_responses
]
problem_type = "bond-court/runif"
model_path = f"models/bond-court/random_order_full_model_0.joblib.gz"
classifier_path = f"classifiers/{problem_type}-classifier-{42}.pkl"
dissonance(processed_responses, problem_type, model_path, classifier_path)

{0: 0.0,
 1: 0.0,
 2: 0.01,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.1,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.02,
 18: 0.02,
 19: 0.0,
 20: 0.0,
 21: 0.0,
 22: 0.0,
 23: 0.0,
 24: 0.0,
 25: 0.0,
 26: 0.0,
 27: 0.0,
 28: 0.0,
 29: 0.0,
 30: 0.0,
 31: 0.0,
 32: 0.0,
 33: 0.0,
 34: 0.0,
 35: 0.0,
 36: 0.0,
 37: 0.0,
 38: 0.0,
 39: 0.0,
 40: 0.0,
 41: 0.0,
 42: 0.0,
 43: 0.0,
 44: 0.02,
 45: 0.0,
 46: 0.0,
 47: 0.0,
 48: 0.0,
 49: 0.0,
 50: 0.0,
 51: 0.0,
 52: 0.0,
 53: 0.0,
 54: 0.0,
 55: 0.0,
 56: 0.0,
 57: 0.0,
 58: 0.0,
 59: 0.0,
 60: 0.0,
 61: 0.03,
 62: 0.0,
 63: 0.0,
 64: 0.0,
 65: 0.0,
 66: 0.0,
 67: 0.0,
 68: 0.0,
 69: 0.0,
 70: 0.0,
 71: 0.0,
 72: 0.0,
 73: 0.0,
 74: 0.0,
 75: 0.0,
 76: 0.0,
 77: 0.0,
 78: 0.0,
 79: 0.0,
 80: 0.0,
 81: 0.0,
 82: 0.0,
 83: 0.0,
 84: 0.2,
 85: 0.0,
 86: 0.0,
 87: 0.0,
 88: 0.0,
 89: 0.0,
 90: 0.01,
 91: 0.0,
 92: 0.0,
 93: 0.0,
 94: 0.0,
 95: 0.0,
 96: 0.0,
 97: 0.0,
 98: 0.0,
 99: 0.0,
 100

In [11]:
patients_responses = (
    pd.read_csv("../data/bondcourt/CCHHS_SUD_details.csv")
    .drop("Interview_ID", axis="columns")
    .fillna(-9)
    .astype(int)
    .replace(-9, "")
    .to_dict("index")
)
processed_responses = [
    {patient: patients_responses[patient]} for patient in patients_responses
]
problem_type = "bond-court/expert"
model_path = f"models/bond-court/random_order_full_model_0.joblib.gz"
classifier_path = f"classifiers/{problem_type}-classifier-{42}.pkl"
dissonance(processed_responses, problem_type, model_path, classifier_path)

{0: 0.0,
 1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.05,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.01,
 16: 0.0,
 17: 0.0,
 18: 0.01,
 19: 0.0,
 20: 0.0,
 21: 0.0,
 22: 0.0,
 23: 0.02,
 24: 0.0,
 25: 0.01,
 26: 0.0,
 27: 0.0,
 28: 0.01,
 29: 0.0,
 30: 0.0,
 31: 0.0,
 32: 0.0,
 33: 0.0,
 34: 0.01,
 35: 0.0,
 36: 0.0,
 37: 0.0,
 38: 0.0,
 39: 0.0,
 40: 0.0,
 41: 0.0,
 42: 0.0,
 43: 0.0,
 44: 0.03,
 45: 0.0,
 46: 0.0,
 47: 0.0,
 48: 0.0,
 49: 0.0,
 50: 0.0,
 51: 0.0,
 52: 0.0,
 53: 0.0,
 54: 0.0,
 55: 0.0,
 56: 0.0,
 57: 0.0,
 58: 0.0,
 59: 0.0,
 60: 0.0,
 61: 0.04,
 62: 0.0,
 63: 0.01,
 64: 0.08,
 65: 0.0,
 66: 0.0,
 67: 0.01,
 68: 0.0,
 69: 0.0,
 70: 0.0,
 71: 0.0,
 72: 0.0,
 73: 0.0,
 74: 0.0,
 75: 0.0,
 76: 0.0,
 77: 0.0,
 78: 0.01,
 79: 0.01,
 80: 0.0,
 81: 0.0,
 82: 0.0,
 83: 0.0,
 84: 0.03,
 85: 0.01,
 86: 0.0,
 87: 0.0,
 88: 0.0,
 89: 0.0,
 90: 0.09,
 91: 0.0,
 92: 0.0,
 93: 0.01,
 94: 0.0,
 95: 0.0,
 96: 0.0,
 97: 0.0,
 98: 0.0,
 9